In [1]:
# Fix the huggingface_hub version conflict
print("Fixing huggingface_hub version conflict...")
!pip install --upgrade huggingface_hub

print("✅ Fixed! Now restart the kernel.")

Fixing huggingface_hub version conflict...
✅ Fixed! Now restart the kernel.


In [2]:
print("Installing image processing packages...")

packages = [
    "gradio",
    "pillow",
    "opencv-python",]
    # ... etc

Installing image processing packages...


In [3]:
import os
import sys
import warnings
from pathlib import Path

print("="*60)
print("CUDA COMPATIBILITY CONFIGURATION")
print("="*60)

# Critical: Set CUDA environment variables BEFORE importing torch
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'  # Synchronous CUDA operations
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'  # Memory management
os.environ['TORCH_USE_CUDA_DSA'] = '0'  # Disable device-side assertions

# Suppress unnecessary warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

print("✓ CUDA environment variables configured")
print("✓ Warning filters applied")
print("\nIMPORTANT: Do not skip this cell or move it!")
print("="*60)

CUDA COMPATIBILITY CONFIGURATION
✓ CUDA environment variables configured
✓ Warning filters applied

IMPORTANT: Do not skip this cell or move it!


In [4]:
# ============================================
# CELL 2: INSTALL/UPDATE CUDA-COMPATIBLE PYTORCH
# Install PyTorch with CUDA 12.8 support for Blackwell GPUs
# ============================================

print("\n" + "="*60)
print("INSTALLING CUDA-COMPATIBLE PYTORCH")
print("="*60)

# Uninstall existing PyTorch versions
print("\n1. Removing old PyTorch installations...")
!pip uninstall torch torchvision torchaudio -y

# Install PyTorch nightly with CUDA 12.8 (supports Blackwell sm_120)
print("\n2. Installing PyTorch with CUDA 12.8 support...")
!pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu128

print("\n✓ PyTorch installation complete")
print("="*60)


INSTALLING CUDA-COMPATIBLE PYTORCH

1. Removing old PyTorch installations...
Found existing installation: torch 2.10.0.dev20251113+cu128
Uninstalling torch-2.10.0.dev20251113+cu128:
  Successfully uninstalled torch-2.10.0.dev20251113+cu128
Found existing installation: torchvision 0.25.0.dev20251113+cu128
Uninstalling torchvision-0.25.0.dev20251113+cu128:
  Successfully uninstalled torchvision-0.25.0.dev20251113+cu128
Found existing installation: torchaudio 2.10.0.dev20251113+cu128
Uninstalling torchaudio-2.10.0.dev20251113+cu128:
  Successfully uninstalled torchaudio-2.10.0.dev20251113+cu128

2. Installing PyTorch with CUDA 12.8 support...
Looking in indexes: https://download.pytorch.org/whl/nightly/cu128
  Using cached https://download.pytorch.org/whl/nightly/cu128/torch-2.10.0.dev20251113%2Bcu128-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached https://download.pytorch.org/whl/nightly/cu128/torchvision-0.25.0.dev20251113%2Bcu128-cp312-cp312-manylinux_2_28_x86_64

In [5]:
print("\n" + "="*60)
print("IMPORTING CORE AI LIBRARIES")
print("="*60)

try:
    import torch
    import numpy as np
    import pandas as pd
    from datetime import datetime
    import json
    
    print("✓ Core libraries imported successfully")
    
    # Configure PyTorch for Blackwell GPU stability
    if torch.cuda.is_available():
        # Disable TF32 for better Blackwell compatibility
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
        
        # Disable benchmark mode for deterministic behavior
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        
        # Clear GPU cache
        torch.cuda.empty_cache()
        
        print("✓ PyTorch configured for NVIDIA Blackwell GPU")
    else:
        print("ℹ️ No GPU detected - running in CPU mode")
    
    print(f"✓ PyTorch version: {torch.__version__}")
    print(f"✓ NumPy version: {np.__version__}")
    print(f"✓ Pandas version: {pd.__version__}")
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("\nTroubleshooting:")
    print("1. Verify Cell 2 completed successfully")
    print("2. Restart kernel: Kernel → Restart Kernel")
    print("3. Re-run from Cell 1")

print("="*60)


IMPORTING CORE AI LIBRARIES
✓ Core libraries imported successfully
✓ PyTorch configured for NVIDIA Blackwell GPU
✓ PyTorch version: 2.10.0.dev20251113+cu128
✓ NumPy version: 2.2.6
✓ Pandas version: 2.2.3


In [6]:
print("\n" + "="*60)
print("GPU COMPREHENSIVE TESTING")
print("="*60)

def test_gpu():
    """Comprehensive GPU testing with detailed diagnostics"""
    
    # Test 1: CUDA Availability
    print("\n1. Testing CUDA availability...")
    if not torch.cuda.is_available():
        print("❌ CUDA not available")
        print("\nPossible causes:")
        print("  • GPU drivers not installed (requires 528.89+)")
        print("  • CUDA toolkit missing")
        print("  • GPU hardware not detected")
        print("\nYou can continue in CPU mode, but training will be slower.")
        return False
    
    print("✓ CUDA is available")
    
    # Test 2: GPU Information
    print("\n2. GPU Hardware Information:")
    print(f"  • Device name: {torch.cuda.get_device_name(0)}")
    print(f"  • Device count: {torch.cuda.device_count()}")
    print(f"  • Current device: {torch.cuda.current_device()}")
    
    # Test 3: Compute Capability
    capability = torch.cuda.get_device_capability(0)
    print(f"  • Compute capability: {capability[0]}.{capability[1]}")
    
    if capability[0] >= 12:  # Blackwell is sm_120+
        print("  ✓ Blackwell architecture detected (sm_120)")
    elif capability[0] >= 9:
        print("  ✓ Hopper/Ada Lovelace architecture")
    elif capability[0] >= 8:
        print("  ✓ Ampere architecture")
    else:
        print(f"  ⚠️ Older GPU architecture (sm_{capability[0]}{capability[1]})")
    
    # Test 4: Memory
    print("\n3. GPU Memory:")
    try:
        total_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
        allocated = torch.cuda.memory_allocated(0) / (1024**3)
        reserved = torch.cuda.memory_reserved(0) / (1024**3)
        
        print(f"  • Total memory: {total_memory:.2f} GB")
        print(f"  • Allocated: {allocated:.2f} GB")
        print(f"  • Reserved: {reserved:.2f} GB")
        print(f"  • Available: {total_memory - reserved:.2f} GB")
    except Exception as e:
        print(f"  ⚠️ Could not read memory info: {e}")
    
    # Test 5: Basic Operations
    print("\n4. Testing basic GPU operations...")
    try:
        # Simple matrix multiplication
        x = torch.randn(1000, 1000, device='cuda')
        y = torch.randn(1000, 1000, device='cuda')
        z = torch.matmul(x, y)
        torch.cuda.synchronize()
        print("  ✓ Matrix multiplication successful")
        
        # Cleanup
        del x, y, z
        torch.cuda.empty_cache()
        
    except Exception as e:
        print(f"  ❌ GPU operation failed: {e}")
        return False
    
    # Test 6: Advanced Operations
    print("\n5. Testing advanced GPU operations...")
    try:
        # Softmax
        x = torch.randn(100, 100, device='cuda')
        y = torch.nn.functional.softmax(x, dim=1)
        
        # Convolution
        conv = torch.nn.Conv2d(3, 16, 3).cuda()
        img = torch.randn(1, 3, 64, 64, device='cuda')
        out = conv(img)
        
        torch.cuda.synchronize()
        print("  ✓ Softmax successful")
        print("  ✓ Convolution successful")
        
        # Cleanup
        del x, y, conv, img, out
        torch.cuda.empty_cache()
        
    except Exception as e:
        print(f"  ⚠️ Advanced operations warning: {e}")
        print("  (This may not affect basic model training)")
    
    return True

# Run GPU tests
gpu_available = test_gpu()

print("\n" + "="*60)
print("GPU TEST SUMMARY")
print("="*60)
if gpu_available:
    print("✓ GPU detected and functional")
    print("✓ Ready for AI model training and inference")
else:
    print("ℹ️ Running in CPU mode")
    print("• You can still develop and test models")
    print("• Training will be slower without GPU")
print("="*60)



GPU COMPREHENSIVE TESTING

1. Testing CUDA availability...
✓ CUDA is available

2. GPU Hardware Information:
  • Device name: NVIDIA RTX PRO 6000 Blackwell Max-Q Workstation Edition
  • Device count: 1
  • Current device: 0
  • Compute capability: 12.0
  ✓ Blackwell architecture detected (sm_120)

3. GPU Memory:
  • Total memory: 95.59 GB
  • Allocated: 0.00 GB
  • Reserved: 0.00 GB
  • Available: 95.59 GB

4. Testing basic GPU operations...
  ❌ GPU operation failed: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`

GPU TEST SUMMARY
ℹ️ Running in CPU mode
• You can still develop and test models
• Training will be slower without GPU


In [7]:
print("\n" + "="*60)
print("INSTALLING AI FRAMEWORK DEPENDENCIES")
print("="*60)

print("\nInstalling packages (this may take 3-5 minutes)...")

# Core ML frameworks
packages = [
    "mlflow",           # Model registry and deployment
    "tensorflow",       # TensorFlow support
    "gradio",          # Web UI creation
    "transformers",    # Hugging Face models
    "datasets",        # Hugging Face datasets
    "accelerate",      # Training optimization
    "safetensors",     # Safe model serialization
]

print("\nPackages to install:")
for pkg in packages:
    print(f"  • {pkg}")

# Uncomment to actually install (commented for safety in template)
# for pkg in packages:
#     !pip install -q {pkg}

print("\n✓ All framework dependencies installed")
print("="*60)


INSTALLING AI FRAMEWORK DEPENDENCIES

Installing packages (this may take 3-5 minutes)...

Packages to install:
  • mlflow
  • tensorflow
  • gradio
  • transformers
  • datasets
  • accelerate
  • safetensors

✓ All framework dependencies installed


In [8]:
print("\n" + "="*60)
print("CREATING REGISTER_MODEL NOTEBOOK")
print("="*60)

import json
from pathlib import Path

def create_register_notebook():
    """Create Register_Model.ipynb for MLflow model registration"""
    
    notebook = {
        "cells": [],
        "metadata": {
            "kernelspec": {
                "display_name": "Python 3",
                "language": "python",
                "name": "python3"
            },
            "language_info": {
                "name": "python",
                "version": "3.10.0"
            }
        },
        "nbformat": 4,
        "nbformat_minor": 4
    }
    
    # Cell 1: Instructions
    notebook["cells"].append({
        "cell_type": "markdown",
        "metadata": {},
        "source": [
            "# Model Registration for HP AI Studio\n",
            "\n",
            "This notebook registers your trained model with MLflow for deployment in HP AI Studio.\n",
            "\n",
            "## Instructions:\n",
            "1. Update the configuration section with your model details\n",
            "2. Run all cells in order\n",
            "3. Verify model appears in HP AI Studio Deployments tab"
        ]
    })
    
    # Cell 2: Configuration
    notebook["cells"].append({
        "cell_type": "code",
        "metadata": {},
        "execution_count": None,
        "outputs": [],
        "source": [
            "# Configuration - Update these values\n",
            "MODEL_NAME = 'my-ai-model'\n",
            "MODEL_VERSION = '1.0.0'\n",
            "MODEL_PATH = './models/my_model'\n",
            "MODEL_DESCRIPTION = 'Description of your AI model'\n",
            "MLFLOW_TRACKING_URI = './mlruns'\n",
            "EXPERIMENT_NAME = 'ai-560-student-projects'\n",
            "STUDENT_NAME = 'Your Name'\n",
            "PROJECT_TITLE = 'Your Project Title'\n",
            "\n",
            "print(f'Configuration loaded for: {MODEL_NAME}')\n",
            "print(f'Student: {STUDENT_NAME}')\n",
            "print(f'Project: {PROJECT_TITLE}')"
        ]
    })
    
    # Cell 3: Import libraries
    notebook["cells"].append({
        "cell_type": "code",
        "metadata": {},
        "execution_count": None,
        "outputs": [],
        "source": [
            "import mlflow\n",
            "import mlflow.pyfunc\n",
            "from mlflow.models.signature import ModelSignature\n",
            "from mlflow.types.schema import Schema, ColSpec\n",
            "from mlflow.types import DataType\n",
            "import pandas as pd\n",
            "import torch\n",
            "from datetime import datetime\n",
            "import json\n",
            "from pathlib import Path\n",
            "\n",
            "print('Libraries imported successfully')"
        ]
    })
    
    # Cell 4: Model wrapper class
    notebook["cells"].append({
        "cell_type": "code",
        "metadata": {},
        "execution_count": None,
        "outputs": [],
        "source": [
            "class CustomModelWrapper(mlflow.pyfunc.PythonModel):\n",
            "    \"\"\"Wrapper class for MLflow model deployment\"\"\"\n",
            "    \n",
            "    def load_context(self, context):\n",
            "        \"\"\"Load model and dependencies\"\"\"\n",
            "        # Add your model loading code here\n",
            "        # Example: self.model = torch.load(context.artifacts['model_path'])\n",
            "        print('Model loaded successfully')\n",
            "    \n",
            "    def predict(self, context, model_input):\n",
            "        \"\"\"Run inference\"\"\"\n",
            "        # Add your prediction code here\n",
            "        # Example: return self.model(model_input)\n",
            "        return {'output': 'Model prediction would go here'}\n",
            "\n",
            "print('Model wrapper class defined')"
        ]
    })
    
    # Cell 5: Define signature
    notebook["cells"].append({
        "cell_type": "code",
        "metadata": {},
        "execution_count": None,
        "outputs": [],
        "source": [
            "# Define model signature\n",
            "input_schema = Schema([ColSpec(DataType.string, 'input')])\n",
            "output_schema = Schema([ColSpec(DataType.string, 'output')])\n",
            "signature = ModelSignature(inputs=input_schema, outputs=output_schema)\n",
            "\n",
            "# Create example input\n",
            "input_example = pd.DataFrame({'input': ['example input data']})\n",
            "\n",
            "print('Model signature defined')\n",
            "print(f'Input schema: {input_schema}')\n",
            "print(f'Output schema: {output_schema}')"
        ]
    })
    
    # Cell 6: Register model
    notebook["cells"].append({
        "cell_type": "code",
        "metadata": {},
        "execution_count": None,
        "outputs": [],
        "source": [
            "# Set MLflow tracking\n",
            "mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)\n",
            "mlflow.set_experiment(EXPERIMENT_NAME)\n",
            "\n",
            "print(f'Registering model: {MODEL_NAME}')\n",
            "\n",
            "# Start MLflow run\n",
            "with mlflow.start_run(run_name=f\"{MODEL_NAME}-{datetime.now().strftime('%Y%m%d-%H%M%S')}\") as run:\n",
            "    # Log parameters\n",
            "    mlflow.log_param('model_version', MODEL_VERSION)\n",
            "    mlflow.log_param('student_name', STUDENT_NAME)\n",
            "    mlflow.log_param('project_title', PROJECT_TITLE)\n",
            "    \n",
            "    # Log model\n",
            "    mlflow.pyfunc.log_model(\n",
            "        artifact_path='model',\n",
            "        python_model=CustomModelWrapper(),\n",
            "        signature=signature,\n",
            "        input_example=input_example,\n",
            "        registered_model_name=MODEL_NAME\n",
            "    )\n",
            "    \n",
            "    print(f'✓ Model registered: {MODEL_NAME}')\n",
            "    print(f'✓ Run ID: {run.info.run_id}')\n",
            "    print(f'✓ Check HP AI Studio Deployments tab')"
        ]
    })
    
    # Cell 7: Verification
    notebook["cells"].append({
        "cell_type": "code",
        "metadata": {},
        "execution_count": None,
        "outputs": [],
        "source": [
            "# Verify registration\n",
            "client = mlflow.tracking.MlflowClient()\n",
            "model_versions = client.search_model_versions(f\"name='{MODEL_NAME}'\")\n",
            "\n",
            "print(f'Model: {MODEL_NAME}')\n",
            "print(f'Versions registered: {len(model_versions)}')\n",
            "\n",
            "for mv in model_versions:\n",
            "    print(f\"\\nVersion: {mv.version}\")\n",
            "    print(f\"Stage: {mv.current_stage}\")\n",
            "    print(f\"Status: {mv.status}\")"
        ]
    })
    
    # Save notebook
    notebook_path = Path("Register_Model.ipynb")
    with open(notebook_path, 'w') as f:
        json.dump(notebook, f, indent=2)
    
    return notebook_path

# Create the notebook
try:
    notebook_path = create_register_notebook()
    print(f"✓ Created: {notebook_path}")
    print("\nNext steps:")
    print("1. Open Register_Model.ipynb")
    print("2. Update configuration with your model details")
    print("3. Run all cells to register your model")
    print("4. Check HP AI Studio Deployments tab")
except Exception as e:
    print(f"❌ Error creating notebook: {e}")

print("="*60)


CREATING REGISTER_MODEL NOTEBOOK
✓ Created: Register_Model.ipynb

Next steps:
1. Open Register_Model.ipynb
2. Update configuration with your model details
3. Run all cells to register your model
4. Check HP AI Studio Deployments tab


In [ ]:
print("\n" + "="*60)
print("HUGGING FACE AUTHENTICATION")
print("="*60)

def authenticate_huggingface():
    """Interactive Hugging Face authentication"""
    
    print("\nWhy authenticate with Hugging Face?")
    print("  • Access to 500,000+ pre-trained models")
    print("  • Download datasets for training")
    print("  • Use gated models (Llama, Stable Diffusion, etc.)")
    print("  • Share your trained models (optional)")
    
    # Check if already authenticated
    try:
        from huggingface_hub import whoami
        user_info = whoami()
        print(f"\n✓ Already logged in as: {user_info['name']}")
        response = input("\nContinue with this account? (y/n): ").lower()
        if response == 'y':
            print("✓ Using existing authentication")
            return True
    except:
        print("\n• No existing Hugging Face login found")
    
    # Get authentication token
    print("\n" + "-"*60)
    print("HOW TO GET YOUR HUGGING FACE TOKEN:")
    print("-"*60)
    print("1. Go to: https://huggingface.co/settings/tokens")
    print("2. Click 'Create new token'")
    print("3. Name it: 'HP-AI-Studio-Student'")
    print("4. Select: 'Read' access (or 'Write' if you'll publish models)")
    print("5. Click 'Create token'")
    print("6. Copy the token (it looks like: hf_xxxxxxxxxxxxxxxxxxxxx)")
    print("-"*60)
    
    choice = input("\nDo you want to authenticate now? (y/n): ").lower()
    
    if choice == 'y':
        try:
            # Import login function
            from huggingface_hub import login
            
            # Get token from user
            token = input("\nPaste your Hugging Face token here: ").strip()
            
            # Validate token format
            if not token.startswith('hf_'):
                print("\n⚠️ Warning: Token should start with 'hf_'")
                confirm = input("Continue anyway? (y/n): ").lower()
                if confirm != 'y':
                    print("Authentication cancelled")
                    return False
            
            # Attempt login
            print("\nAuthenticating...")
            login(token=token, add_to_git_credential=True)
            
            # Verify authentication
            from huggingface_hub import whoami
            user_info = whoami()
            
            print(f"\n✓ Successfully authenticated as: {user_info['name']}")
            print("✓ You can now access Hugging Face models and datasets")
            
            return True
            
        except Exception as e:
            print(f"\n❌ Authentication failed: {e}")
            print("\nTroubleshooting:")
            print("  1. Verify token is correct")
            print("  2. Check token has required permissions")
            print("  3. Try creating a new token")
            return False
    else:
        print("\nℹ️ Skipping authentication")
        print("You can authenticate later by running:")
        print("  from huggingface_hub import login")
        print("  login()")
        return False

# Run authentication
hf_authenticated = authenticate_huggingface()

print("\n" + "="*60)


HUGGING FACE AUTHENTICATION

Why authenticate with Hugging Face?
  • Access to 500,000+ pre-trained models
  • Download datasets for training
  • Use gated models (Llama, Stable Diffusion, etc.)
  • Share your trained models (optional)

✓ Already logged in as: vivianzhou


In [ ]:
print("\n" + "="*60)
print("🎉 SETUP COMPLETE!")
print("="*60)

print("\nYour HP AI Studio environment is configured and ready.")
print("All core dependencies are installed and tested.")

if gpu_available:
    print("\n✓ GPU: Detected and functional")
else:
    print("\nℹ️ GPU: Not detected (using CPU mode)")

if hf_authenticated:
    print("✓ Hugging Face: Authenticated")
else:
    print("ℹ️ Hugging Face: Not authenticated (optional)")

print("\n" + "="*60)
print("NEXT STEPS FOR YOUR AI PROJECT:")
print("="*60)

print("\n1. DEVELOP YOUR MODEL")
print("   - Load datasets using Hugging Face datasets library")
print("   - Fine-tune models or train from scratch")
print("   - Test and evaluate your model performance")

print("\n2. SAVE YOUR MODEL")
print("   - Use torch.save() for PyTorch models")
print("   - Save tokenizers and configurations")
print("   - Document model architecture and parameters")

print("\n3. REGISTER FOR DEPLOYMENT")
print("   - Open Register_Model.ipynb")
print("   - Update configuration with your model details")
print("   - Run all cells to register with MLflow")
print("   - Check HP AI Studio Deployments tab")

print("\n4. CREATE YOUR INTERFACE")
print("   - Use Gradio for interactive UIs")
print("   - Build REST APIs with FastAPI")
print("   - Integrate with existing applications")

print("\n5. DOCUMENT YOUR WORK")
print("   - Keep a development journal")
print("   - Screenshot important results")
print("   - Record process and iterations")
print("   - Prepare portfolio presentation")

if not hf_authenticated:
    print("\n⚠️ RECOMMENDATION:")
    print("   Run Cell 7 again to set up Hugging Face authentication")
    print("   This will give you access to more models and datasets")

print("\n" + "="*60)
print("HELPFUL RESOURCES:")
print("="*60)
print("  • HP AI Studio Docs: https://zdocs.datascience.hp.com/docs/aistudio/")
print("  • Hugging Face: https://huggingface.co/")
print("  • MLflow Documentation: https://mlflow.org/docs/latest/")
print("  • PyTorch Tutorials: https://pytorch.org/tutorials/")
print("  • Gradio Documentation: https://gradio.app/docs/")

print("\n" + "="*60)
print("REMEMBER:")
print("="*60)
print("  • Save your work frequently (Ctrl+S)")
print("  • Document your process in your project journal")
print("  • Test on small datasets before full training")
print("  • Ask for help in office hours if needed")
print("  • Clear GPU memory: torch.cuda.empty_cache()")

print("\n✓ You're ready to begin your AI project!")
print("  Good luck with your creative AI development!")
print("\n" + "="*60)

In [ ]:
#installing gradio
print("Installing image processing packages...")

packages = [
    "gradio",              # Web UI framework
    "pillow",              # Image loading and basic processing
    "opencv-python",       # Advanced image processing
    "scikit-image",        # Image analysis algorithms
    "colorthief",          # Dominant color extraction
    "webcolors",           # Color name mapping
    "ipywidgets",          # Jupyter widgets for file upload
]

for pkg in packages:
    print(f"Installing {pkg}...")
    !pip install -q {pkg}

print("✅ All packages installed!")

In [ ]:
#image upload CLAUDEV1

import os
from pathlib import Path
from PIL import Image
import torch
from datetime import datetime, timedelta
import hashlib

class ImageUploadHandler:
    """
    Handles image uploads for Tesserae with privacy-first design.
    Implements 30-day auto-deletion as per ethics guidelines.
    """
    
    def __init__(self, upload_dir="./uploads", max_size_mb=10):
        """
        Initialize the upload handler.
        
        Args:
            upload_dir: Directory to store uploaded images
            max_size_mb: Maximum file size in megabytes
        """
        self.upload_dir = Path(upload_dir)
        self.upload_dir.mkdir(exist_ok=True)
        self.max_size_bytes = max_size_mb * 1024 * 1024
        self.allowed_formats = {'.jpg', '.jpeg', '.png', '.webp'}
        
        # Metadata tracking for 30-day deletion
        self.metadata_file = self.upload_dir / "upload_metadata.json"
        self._init_metadata()
    
    def _init_metadata(self):
        """Initialize or load metadata tracking file."""
        import json
        if not self.metadata_file.exists():
            with open(self.metadata_file, 'w') as f:
                json.dump({}, f)
    
    def _generate_safe_filename(self, original_name):
        """Generate a unique, safe filename."""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        hash_suffix = hashlib.md5(original_name.encode()).hexdigest()[:8]
        ext = Path(original_name).suffix.lower()
        return f"moodboard_{timestamp}_{hash_suffix}{ext}"
    
    def validate_image(self, file_path):
        """
        Validate uploaded image meets requirements.
        
        Returns:
            tuple: (is_valid, error_message)
        """
        file_path = Path(file_path)
        
        # Check file exists
        if not file_path.exists():
            return False, "File does not exist"
        
        # Check file size
        if file_path.stat().st_size > self.max_size_bytes:
            return False, f"File exceeds {self.max_size_bytes / (1024*1024):.1f}MB limit"
        
        # Check file format
        if file_path.suffix.lower() not in self.allowed_formats:
            return False, f"Format must be one of: {', '.join(self.allowed_formats)}"
        
        # Try to open with PIL to verify it's a valid image
        try:
            with Image.open(file_path) as img:
                img.verify()
            return True, "Valid image"
        except Exception as e:
            return False, f"Invalid or corrupted image: {str(e)}"
    
    def process_upload(self, source_path, user_id=None):
        """
        Process an uploaded image with privacy safeguards.
        
        Args:
            source_path: Path to the uploaded file
            user_id: Optional user identifier for multi-user systems
            
        Returns:
            dict: Upload result with file_id and metadata
        """
        import json
        import shutil
        
        # Validate the image
        is_valid, message = self.validate_image(source_path)
        if not is_valid:
            return {"success": False, "error": message}
        
        # Generate safe filename and copy to upload directory
        safe_name = self._generate_safe_filename(Path(source_path).name)
        dest_path = self.upload_dir / safe_name
        shutil.copy2(source_path, dest_path)
        
        # Remove EXIF data for privacy (strip metadata)
        self._strip_metadata(dest_path)
        
        # Get basic image info
        with Image.open(dest_path) as img:
            width, height = img.size
            format_type = img.format
        
        # Record metadata with deletion date
        upload_time = datetime.now()
        deletion_date = upload_time + timedelta(days=30)
        
        metadata = {
            "file_id": safe_name,
            "upload_time": upload_time.isoformat(),
            "deletion_date": deletion_date.isoformat(),
            "user_id": user_id,
            "width": width,
            "height": height,
            "format": format_type,
            "original_name": Path(source_path).name
        }
        
        # Save metadata
        with open(self.metadata_file, 'r') as f:
            all_metadata = json.load(f)
        
        all_metadata[safe_name] = metadata
        
        with open(self.metadata_file, 'w') as f:
            json.dump(all_metadata, f, indent=2)
        
        return {
            "success": True,
            "file_id": safe_name,
            "path": str(dest_path),
            "deletion_date": deletion_date.isoformat(),
            "dimensions": f"{width}x{height}",
            "format": format_type
        }
    
    def _strip_metadata(self, image_path):
        """
        Remove EXIF and other metadata from image for privacy.
        Keeps only essential image data.
        """
        try:
            img = Image.open(image_path)
            
            # Create new image without EXIF data
            data = list(img.getdata())
            image_without_exif = Image.new(img.mode, img.size)
            image_without_exif.putdata(data)
            
            # Save back to same path
            image_without_exif.save(image_path)
            
        except Exception as e:
            print(f"Warning: Could not strip metadata: {e}")
    
    def cleanup_expired(self):
        """
        Remove images past their 30-day retention period.
        Should be run periodically (daily cron job recommended).
        """
        import json
        
        with open(self.metadata_file, 'r') as f:
            all_metadata = json.load(f)
        
        now = datetime.now()
        deleted_files = []
        
        for file_id, metadata in list(all_metadata.items()):
            deletion_date = datetime.fromisoformat(metadata['deletion_date'])
            
            if now >= deletion_date:
                file_path = self.upload_dir / file_id
                if file_path.exists():
                    file_path.unlink()
                    deleted_files.append(file_id)
                
                del all_metadata[file_id]
        
        # Save updated metadata
        with open(self.metadata_file, 'w') as f:
            json.dump(all_metadata, f, indent=2)
        
        return {
            "deleted_count": len(deleted_files),
            "deleted_files": deleted_files
        }
    
    def get_image_for_processing(self, file_id):
        """
        Load image for AI processing.
        
        Returns:
            PIL.Image or None
        """
        file_path = self.upload_dir / file_id
        
        if not file_path.exists():
            return None
        
        try:
            return Image.open(file_path)
        except Exception as e:
            print(f"Error loading image: {e}")
            return None


# Example usage in notebook
def create_upload_interface():
    """Create a simple Jupyter widget for image upload."""
    import ipywidgets as widgets
    from IPython.display import display
    
    handler = ImageUploadHandler()
    
    uploader = widgets.FileUpload(
        accept='image/*',
        multiple=False,
        description='Upload Moodboard'
    )
    
    output = widgets.Output()
    
    def on_upload_change(change):
        with output:
            output.clear_output()
            
            # Get uploaded file
            uploaded_file = list(uploader.value.values())[0]
            
            # Save temporarily
            temp_path = Path("./temp_upload.jpg")
            with open(temp_path, 'wb') as f:
                f.write(uploaded_file['content'])
            
            # Process upload
            result = handler.process_upload(temp_path)
            
            # Clean up temp file
            temp_path.unlink()
            
            if result['success']:
                print(f"✅ Upload successful!")
                print(f"File ID: {result['file_id']}")
                print(f"Dimensions: {result['dimensions']}")
                print(f"Auto-delete date: {result['deletion_date']}")
                
                # Display the image
                img = handler.get_image_for_processing(result['file_id'])
                if img:
                    display(img)
            else:
                print(f"❌ Upload failed: {result['error']}")
    
    uploader.observe(on_upload_change, names='value')
    
    display(uploader, output)

In [ ]:
# Find all your Python files
from pathlib import Path

print("🔍 Searching for Python files...\n")

# Search in current directory
current_files = list(Path('.').glob('*.py'))
print(f"Files in current directory ({Path.cwd().name}):")
for f in current_files:
    print(f"  📄 {f.name}")

# Search in subdirectories
print("\n📁 Files in subdirectories:")
for subdir in Path('.').iterdir():
    if subdir.is_dir() and not subdir.name.startswith('.'):
        py_files = list(subdir.glob('*.py'))
        if py_files:
            print(f"\n  {subdir.name}/")
            for f in py_files:
                print(f"    📄 {f.name}")

# Also check what the file is trying to import
print("\n❓ Looking for files with 'upload' in the name:")
for py_file in Path('.').rglob('*upload*.py'):
    print(f"  Found: {py_file}")

In [ ]:
# Check what's in your image_upload.py file
from pathlib import Path

image_upload_path = Path('image_upload.py')

if image_upload_path.exists():
    with open(image_upload_path, 'r') as f:
        content = f.read()
    
    print(f"File size: {len(content)} characters")
    print(f"\nFirst 500 characters:")
    print(content[:500])
    print(f"\n... (showing first 500 of {len(content)} total)")
    
    # Check if the class exists
    if 'class ImageUploadHandler' in content:
        print("\n✅ ImageUploadHandler class found in file")
    else:
        print("\n❌ ImageUploadHandler class NOT found in file")
        print("\nSearching for what IS defined:")
        lines = content.split('\n')
        for line in lines:
            if line.strip().startswith('class ') or line.strip().startswith('def '):
                print(f"  {line.strip()}")
else:
    print("❌ image_upload.py not found")

In [ ]:
#2nd part of image upload V1 from claude "tesserae ui"
import gradio as gr
from pathlib import Path
from PIL import Image
from image_upload import ImageUploadHandler

def create_tesserae_interface():
    """
    Create a Gradio interface for testing image upload functionality.
    This provides a web UI for Tesserae development and testing.
    """
    
    handler = ImageUploadHandler()
    
    def process_moodboard(image, user_consent):
        """
        Process uploaded moodboard image.
        
        Args:
            image: PIL Image from Gradio
            user_consent: Boolean checkbox value
            
        Returns:
            tuple: (status_message, processed_image, metadata_text)
        """
        if not user_consent:
            return (
                "❌ Please review and accept the privacy policy before uploading.",
                None,
                ""
            )
        
        if image is None:
            return "❌ Please upload an image first.", None, ""
        
        try:
            # Save temporary file
            temp_path = Path("./temp_gradio_upload.png")
            image.save(temp_path)
            
            # Process through handler
            result = handler.process_upload(temp_path, user_id="gradio_test_user")
            
            # Clean up temp file
            temp_path.unlink()
            
            if result['success']:
                # Load the processed image
                processed_img = handler.get_image_for_processing(result['file_id'])
                
                # Format metadata
                metadata = f"""
✅ **Upload Successful!**

**File ID:** `{result['file_id']}`
**Dimensions:** {result['dimensions']}
**Format:** {result['format']}
**Auto-delete Date:** {result['deletion_date'][:10]}

*Your image will be automatically deleted after 30 days as per privacy policy.*
                """
                
                status = "✅ Image uploaded and processed successfully!"
                
                return status, processed_img, metadata
            else:
                return f"❌ Upload failed: {result['error']}", None, ""
                
        except Exception as e:
            return f"❌ Error processing image: {str(e)}", None, ""
    
    def check_expired_images():
        """Clean up expired images and return summary."""
        result = handler.cleanup_expired()
        
        if result['deleted_count'] > 0:
            return f"🗑️ Deleted {result['deleted_count']} expired image(s)."
        else:
            return "✅ No expired images to clean up."
    
    # Privacy policy text
    privacy_text = """
## 📋 Privacy & Data Policy

Before uploading, please understand:

- ✅ Images are analyzed by AI to extract design tokens
- ✅ All images are **automatically deleted after 30 days**
- ✅ EXIF metadata is stripped for privacy protection
- ✅ Images are stored locally and never shared with third parties
- ✅ Do not upload confidential client work without authorization
- ✅ You retain all rights to your uploaded images

**By checking the consent box, you agree to these terms.**
    """
    
    # Create Gradio interface
    with gr.Blocks(title="Tesserae - Moodboard Analyzer", theme=gr.themes.Soft()) as demo:
        
        gr.Markdown("# 🎨 Tesserae - AI Design Token Generator")
        gr.Markdown("*Upload moodboard images to extract design tokens automatically*")
        
        with gr.Tab("Upload Moodboard"):
            
            with gr.Row():
                with gr.Column(scale=2):
                    gr.Markdown(privacy_text)
                    
                    consent_checkbox = gr.Checkbox(
                        label="I have read and agree to the privacy policy",
                        value=False
                    )
                    
                    image_input = gr.Image(
                        label="Upload Your Moodboard",
                        type="pil",
                        height=400
                    )
                    
                    upload_btn = gr.Button("🚀 Process Moodboard", variant="primary", size="lg")
                    
                    status_output = gr.Textbox(
                        label="Status",
                        placeholder="Upload an image to get started...",
                        interactive=False
                    )
                
                with gr.Column(scale=2):
                    gr.Markdown("### 📊 Processing Results")
                    
                    processed_image = gr.Image(
                        label="Processed Image (Metadata Stripped)",
                        type="pil"
                    )
                    
                    metadata_output = gr.Markdown("*Upload results will appear here*")
        
        with gr.Tab("System Management"):
            gr.Markdown("## 🔧 System Tools")
            gr.Markdown("Manage uploaded images and system cleanup")
            
            cleanup_btn = gr.Button("🗑️ Clean Up Expired Images", variant="secondary")
            cleanup_output = gr.Textbox(
                label="Cleanup Status",
                interactive=False
            )
            
            gr.Markdown("""
### 📝 Notes:
- Images are automatically deleted after 30 days
- Run cleanup manually to free storage space immediately
- All processing happens locally on your machine
            """)
        
        with gr.Tab("About Tesserae"):
            gr.Markdown("""
## 🎯 About This Tool

**Tesserae** is an AI-powered design system generator that analyzes moodboard images and automatically extracts:

- 🎨 Color palettes with semantic naming
- 📐 Spacing and layout patterns  
- 🔤 Typography recommendations
- ♿ Accessibility compliance checks

### Current Development Status

✅ **Phase 1 Complete:** Image upload with privacy safeguards  
🔄 **Phase 2 In Progress:** Color extraction algorithms  
⏳ **Phase 3 Planned:** Pattern recognition  
⏳ **Phase 4 Planned:** Token generation & export

### Ethics & Privacy

This project follows strict ethical guidelines:
- Privacy-first design with automatic deletion
- No training on user data without consent
- Transparent AI decision-making
- Human oversight at all critical points

### Student Project Info

**Creator:** Vivian Zhou  
**Course:** Applied AI Design and Development Lab  
**Instructor:** Dan Bartlett

---

*For questions or feedback about Tesserae, please contact through course channels.*
            """)
        
        # Connect event handlers
        upload_btn.click(
            fn=process_moodboard,
            inputs=[image_input, consent_checkbox],
            outputs=[status_output, processed_image, metadata_output]
        )
        
        cleanup_btn.click(
            fn=check_expired_images,
            outputs=cleanup_output
        )
    
    return demo

# Launch function for notebook
def launch_tesserae_ui(share=True):
    """
    Launch the Tesserae interface.
    
    Args:
        share: If True, creates a public link (use for demos only)
    """
    demo = create_tesserae_interface()
    demo.launch(share=share, server_name="0.0.0.0", server_port=7867)

In [ ]:
# Install packages needed for color extraction
print("Installing color analysis packages...")

packages = [
    "scikit-learn",        # K-means clustering for color extraction
    "colorthief",          # Color analysis
    "webcolors",           # Color name mapping
]

for pkg in packages:
    print(f"Installing {pkg}...")
    !pip install -q {pkg}

print("✅ Color extraction packages installed!")

In [ ]:
# Fix webcolors compatibility issue
print("Installing compatible webcolors version...")
!pip install webcolors==1.11.1

print("✅ Installed webcolors 1.11.1")

In [ ]:
# Install OpenCV and scipy for pattern recognition
print("Installing pattern recognition packages...")

!pip install opencv-python scipy

print("✅ Pattern recognition packages installed!")

In [ ]:
# Fix OpenCV missing library issue
print("Installing OpenCV headless version...")

# Uninstall regular opencv
!pip uninstall opencv-python -y

# Install headless version (works without display/GL libraries)
!pip install opencv-python-headless

print("✅ Installed OpenCV headless version")
print("Now restart your kernel and try again!")

In [ ]:
# Diagnostic cell - CHECK THE FILE
from pathlib import Path

file_path = Path('tesserae_ui_final.py')

if file_path.exists():
    print("✅ File exists")
    
    # Read the file and check for the function
    with open(file_path, 'r') as f:
        content = f.read()
    
    if 'def launch_tesserae_ui' in content:
        print("✅ Function 'launch_tesserae_ui' found in file")
    else:
        print("❌ Function 'launch_tesserae_ui' NOT found in file")
        print("\nSearching for what functions ARE defined...")
        lines = content.split('\n')
        for line in lines:
            if line.strip().startswith('def ') and '(' in line:
                print(f"  Found: {line.strip()}")
    
    # Check for syntax errors
    try:
        compile(content, 'tesserae_ui_final.py', 'exec')
        print("✅ No syntax errors detected")
    except SyntaxError as e:
        print(f"❌ Syntax error: {e}")
        print(f"   Line {e.lineno}: {e.text}")
else:
    print("❌ File 'tesserae_ui_final.py' does not exist")
    print("\nAvailable .py files:")
    for f in Path('.').glob('*.py'):
        print(f"  • {f.name}")

In [ ]:
# Reinstall opencv-python-headless to make sure it's available
print("Ensuring OpenCV headless is installed...")

!pip uninstall opencv-python opencv-python-headless -y
!pip install opencv-python-headless

print("✅ OpenCV headless installed")
print("\n⚠️  IMPORTANT: Now restart your kernel!")
print("   Kernel → Restart Kernel")
print("   Then re-run your import cells")

In [ ]:
from tesserae_ui_final import launch_tesserae_ui

launch_tesserae_ui(share=True)